In [1]:
import numpy as np
import pandas as pd
import sys
import os
import matplotlib.pyplot as plt
import texttable as tt

import tensorflow as tf
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Activation, Input, Dropout, Add
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam

Using TensorFlow backend.


In [2]:
def load_imgs(data_dir):
    animal_to_imgs = {}
    for animal_name in os.listdir(data_dir):
        animal_to_imgs[animal_name] = []
        animal_dir = data_dir + "/" + animal_name + "/"
        for img_name in os.listdir(animal_dir):
            img = plt.imread(animal_dir + img_name)
            animal_to_imgs[animal_name].append(img)
    return animal_to_imgs

def load_info():
    df_classes = pd.read_csv("classes.txt", header=None) 
    df_predicate_matrix = pd.read_csv("predicate-matrix-binary.txt", header=None)
    df_test_classes = pd.read_csv("testclasses.txt", header=None)
    df_train_classes = pd.read_csv("trainclasses.txt", header=None)

    animal_to_feat = {}
    id_to_name, name_to_id = {}, {}
    for i, c in enumerate(df_classes[0]):
        c_name = c.split()[1]
        id_to_name[i] = c_name
        name_to_id[c_name] = i
        animal_to_feat[c_name] = np.array([int(binary) for binary in df_predicate_matrix.iloc[i, 0].split()])

    train_classes, test_classes = [], []
    for c in df_train_classes[0]: train_classes.append(c.split()[0])
    for c in df_test_classes[0]: test_classes.append(c.split()[0])

    return animal_to_feat, id_to_name, name_to_id, train_classes, test_classes

def pred_class(model, img, classes):
    s = model.predict(np.expand_dims(img, axis=0))[0]
    probs = np.zeros(len(classes))
    for i, animal in enumerate(classes):
        probs[i] = np.prod(np.abs(s - 1.0 + animal_to_feat[animal]))
    return probs.argsort()[-1]

def pred_class_ham(model, img, classes):
    s = np.round(model.predict(np.expand_dims(img, axis=0))[0]).astype(int)
    score = np.zeros(len(classes))
    for i, animal in enumerate(classes):
        score[i] = np.sum(np.abs(s - animal_to_feat[animal]))
    return score.argsort()[0]

def pred_class_sum(model, img, classes):
    s = model.predict(np.expand_dims(img, axis=0))[0]
    probs = np.zeros(len(classes))
    for i, animal in enumerate(classes):
        probs[i] = np.sum(np.abs(s - animal_to_feat[animal]))
    return probs.argsort()[0]

def pred_class_harm(model, img, classes):
    eps = 1e-5
    s = model.predict(np.expand_dims(img, axis=0))[0]
    probs = np.zeros(len(classes))
    for i, animal in enumerate(classes):
        pos = np.sum(np.log(eps + np.abs(s - 1.0 + animal_to_feat[animal])))
        neg = np.sum(np.log(eps + np.abs(s - animal_to_feat[animal])))
        probs[i] = pos - neg
    return probs.argsort()[-1]

def predictions(model, classes, animal_to_images, pred_func=pred_class):
    y_pred, y_true = [], []
    for i, animal in enumerate(classes):
        for img in animal_to_images[animal]:
            y_true.append(i)
            y_pred.append(pred_func(model, img, classes))
    return y_pred, y_true

def pred_features(model, img):
    return np.round(model.predict(np.expand_dims(img, axis=0))[0]).astype(int)

def feature_preds(model, classes, animal_to_images):
    y_pred, y_true = [], []
    for animal in classes:
        for img in animal_to_images[animal]:
            y_true.append(animal_to_feat[animal])
            y_pred.append(pred_features(model, img))
    return y_pred, y_true

def draw_table(scores, predicates):
    table = tt.Texttable()
    table.set_cols_align(["l", "r", "l", "r", "l", "r", "l", "r", "l", "r"])
    table.set_cols_valign(["m", "m", "m", "m", "m", "m", "m", "m", "m", "m"])
    table.set_cols_width([8, 8, 8, 8, 8, 8, 8, 8, 8, 8])
    header = ["Feature", "Score", "Feature", "Score", "Feature", "Score", "Feature", "Score", "Feature", "Score"]
    rows = [header]
    for i in range(0, len(scores), 5):
        temp = []
        for j in range(5): 
            temp.append(predicates[i + j])
            temp.append(round(scores[i + j], 3))
        rows.append(temp)
    table.add_rows(rows)
    print(table.draw())

In [3]:
animal_to_imgs = load_imgs("images_128x128")
animal_to_feat, id_to_name, name_to_id, train_classes, test_classes = load_info()
all_classes = train_classes + test_classes
predicate_file = pd.read_csv("predicates.txt", header=None)
predicates = []
for line in predicate_file.iloc[:,0]: predicates.append(line.split()[-1])

In [7]:
model2 = load_model("models/final-model-90.hdf5")

In [8]:
y_pred, y_test = feature_preds(model2, test_classes, animal_to_imgs)

In [41]:
def f1_score(y_pred, y_test, index):
    pred, test = y_pred[:, index], y_test[:, index]
    d1 = [test[i] for i in range(len(pred)) if pred[i] > 0]
    d2 = [pred[i] for i in range(len(pred)) if test[i] > 0]
    if len(d1) == 0 or len(d2) == 0:
        return 0.0
    p, r = np.mean(d1), np.mean(d2)
    return 2 * p * r / (p + r)

In [42]:
f1_scores = np.zeros(len(predicates))
for i in range(len(predicates)): 
    f1_scores[i] = f1_score(np.array(y_pred), np.array(y_test), i)
print(f1_scores)

[0.71024352 0.51288969 0.42555332 0.63436019 0.67524729 0.
 0.         0.5325779  0.182562   0.14152781 0.00309119 0.78792469
 0.59521051 0.68438146 0.61142576 0.59463851 0.59657702 0.40273738
 0.70978821 0.4825784  0.30136986 0.3533437  0.69431078 0.38637308
 0.         0.76865392 0.75634049 0.53310551 0.20191286 0.49283403
 0.         0.64401246 0.         0.44006501 0.         0.
 0.7192053  0.31672204 0.83305682 0.80792152 0.41145247 0.6578274
 0.3826742  0.58462049 0.36587945 0.88007484 0.62433511 0.53959732
 0.3713885  0.30109268 0.68770137 0.58438547 0.49214868 0.72081218
 0.57351117 0.14966887 0.66424101 0.08540373 0.37279397 0.17715959
 0.27912621 0.28590786 0.87375746 0.91334785 0.58970454 0.64712269
 0.         0.14974705 0.35426429 0.52967359 0.41597971 0.37978142
 0.33642931 0.71129477 0.79731458 0.7205298  0.45389722 0.
 0.46239273 0.74425115 0.80973021 0.58803191 0.66759635 0.47160366
 0.39799966]


In [44]:
test_classes

['chimpanzee',
 'giant+panda',
 'leopard',
 'persian+cat',
 'pig',
 'hippopotamus',
 'humpback+whale',
 'raccoon',
 'rat',
 'seal']

In [43]:
draw_table(f1_scores, predicates)

+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+
| Feature  |  Score   | Feature  |  Score   | Feature  |  Score   | Feature  |  Score   | Feature  |  Score   |
+==========+==========+==========+==========+==========+==========+==========+==========+==========+==========+
| black    |    0.710 | white    |    0.513 | blue     |    0.426 | brown    |    0.634 | gray     |    0.675 |
+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+
| orange   |        0 | red      |        0 | yellow   |    0.533 | patches  |    0.183 | spots    |    0.142 |
+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+
| stripes  |    0.003 | furry    |    0.788 | hairless |    0.595 | toughski |    0.684 | big      |    0.611 |
|          |          |          |          |          |          | n        |          |          |    